In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy import stats

## task 01

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:

ковариацию этих двух величин,
коэффициент корреляции Пирсона.
Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [18]:
salary = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110])
scoring = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])

In [19]:
salary_mean = sum(salary)/len(salary)
scoring_mean = sum(scoring)/len(scoring)

In [20]:
salary_std = (sum([(itm - salary_mean)**2 for itm in salary]) / (len(salary) - 1)) ** 0.5
scoring_std = (sum([(itm - scoring_mean)**2 for itm in scoring]) / (len(scoring) - 1)) ** 0.5

In [21]:
len(salary) == len(scoring)

True

In [22]:
cov = ((salary - salary_mean) * (scoring - scoring_mean)).sum() / (len(salary) - 1)
cov

10175.37777777778

In [23]:
np.cov(salary, scoring, ddof=1)

array([[ 3882.93333333, 10175.37777778],
       [10175.37777778, 33854.32222222]])

In [24]:
corr = cov / (salary_std * scoring_std)
corr

0.8874900920739164

In [25]:
np.corrcoef(salary, scoring)

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

## task 02

Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. Что для этого нужно знать:

Нулевая гипотеза: реальный коэффициент корреляции равен 0. Альтернативная гипотеза двухсторонняя.
Статистика: t = r * sqrt(n - 2) / sqrt(1 - r ** 2), где r - коэффициент корреляции Пирсона, посчитанный по выборке.
В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром df = n - 2

In [26]:
alpha = 0.05
n = len(salary)

In [31]:
def statistic(x: np.ndarray, y: np.ndarray) -> float:
    if len(x) == len(y):
        cov = ((x - x.mean()) * (y - y.mean())).sum() / (len(x) - 1)
        corr = cov / (x.std(ddof = 1) * y.std(ddof = 1))
        return (corr * (len(x) - 2) ** 0.5 / (1 - corr ** 2) ** 0.5)

In [37]:
def critical_borders (alpha, n):

    t1 = stats.t.ppf(alpha / 2, df = n - 2)
    t2 = stats.t.ppf(1 - alpha / 2, df = n - 2)

    return t1, t2

In [33]:
t = statistic(salary, scoring)
t

5.447168150485581

In [38]:
t1, t2 = critical_borders(alpha, n)
t1, t2

(-2.306004135033371, 2.3060041350333704)

In [39]:
t1 < t < t2

False

Значение статистики попало в критическую область, значит, H0 (реальный коэффициент корреляции равен 0) можно отвергнуть при alpha = 0.05

In [35]:
p_left = stats.t.cdf(t, df = n - 2)
p_right = 1 - stats.t.cdf(t, df = n - 2)

pvalue = 2 * min(p_left, p_right)

pvalue

0.0006107546587257318

Нулевую гипотезу можно принять только при альфа < 0.0006107546587257318, при таком значениии это практически нулевая вероятность того, что гипотеза верна

## task 03

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111
Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

In [40]:
iq = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])

In [50]:
n = len(iq)
mean = iq.mean()
std  = iq.std(ddof = 1)
n, mean, std

(10, 118.1, 10.54566788359614)

In [51]:
p = 0.95
alpha = 1 - p

In [52]:
t1 = stats.t.ppf(alpha / 2, df = n - 1)
t2 = stats.t.ppf(1 - alpha / 2, df = n - 1)

t1, t2

(-2.2621571627409915, 2.2621571627409915)

In [53]:
(mean + t1 * std / np.sqrt(n), mean + t2 * std / np.sqrt(n))

(110.55608365158724, 125.64391634841274)

## task 04

**Dataset description:**
синтетический датасет, используемый для тестовых заданий аналитикам.
В нем содержатся данные об установках и всех платежах, совершаемых пользователями во всех приложениях и на всех платформах. Таблица содержит следующие колонки, необходимые для решения задачи:
* Game_ID - название приложения. Примеры: “Hidden City”, “Jewels of Rome”, “Mahjong Journey”.
* Platform - платформа, на которой играет пользователь. Примеры: “Android”, “Facebook”, “iOS”.
* Device_ID - уникальный идентификатор устройства пользователя. Пример: “14321DLVRF5636”.
* Install_DateTime - дата и время установки (первого запуска) приложения. Пример: “2019-12-01 15:54:12”.
* Payment_DayTime - дата и время совершения платежа. Пример: “2019-12-01 15:54:12”.
* Payment_Amount - сумма совершенного платежа в USD. Пример: “49.99”.

Сделаем допущение, что в таблице находятся данные о платежах к 4му дню игры, то есть у каждого last activity day = install day + 4, лайфтайм когорты 4 дня

**Что планирую сделать**

* Добавить в датасет Total Payments для каждого пользователя (по играм разбивать не буду, в данном датасете каждый пользователь платит ровно в одной игре)
* Проверить распределение Total Payments на нормальность
* Проверить гипотезу о том, что мат ожидание Total Payments одинаковое во всех играх
* Проанализировать влияние платформы на Total Payments (можно в одной игре)
* Проверить на корреляцию Total Payments и временной промежуток между инсталлом и датой последнего платежа
